# Audience
Пример отчета по расчету аудитории интернет-ресурсов (Сross Web).

## Описание задачи

Посчитаем аудиторию в разбивке на типы пользования интернетом:
- данные по одному ресурсу Avito;
- данные по двум ресурсам Avito, Ivi, т.е. количество людей, которые посетили хотя бы один ресурс Avito OR Ivi;
- данные по Total Internet

Общие параметры:
- Период: Сентябрь 2021
- География: Россия 0+
- Население: 12+
- Тип пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Статистики:
- Reach (reach)
- Reach% (reachPer)
- Average Daily Reach (adr)
- Average Daily Reach% (adrPer)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Формируем задания

Необходимо сформировать три задания для API Cross Web:
- расчет аудитории ресурса Avito
- расчет совокупной аудитории ресурсов Avito, Ivi
- расчет аудитории Total Internet

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2021-09-01', '2021-09-30')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

### Получим ID ресурсов
Для построения отчета необходимо получить идентификаторы ресурсов __Avito__ и __Ivi__.

Для этого воспользуемся методами поиска в медиа справочнике. Работа с медиа справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

Получим идентификатор ресурса **Avito**

In [ ]:
cats.get_resource(resource='Avito')

Получим идентификатор ресурса **Ivi**

In [ ]:
cats.get_resource(resource='Ivi')

Таким образом, необходимые идентификаторы следующие:

- **Avito**  resourceId = 1028
- **Ivi**  resourceId = 1067

## Задания

Перейдем к формированию заданий.


### Задание №1. Расчет аудитории ресурса Avito

In [ ]:
# Задаем название ресурса для отображения в DataFrame
project_name = 'Avito'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр по медиа, в нашем случае это ID ресурса Avito
mart_filter = 'crossMediaResourceId = 1028'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "useTypeName"]

# Указываем список статистик для расчета
statistics = ['reach', 'reachPer', 'adr', 'adrPer'] 

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                                      demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_avito = mtask.result2table(mtask.get_result(task_audience), project_name = project_name)
df_avito

### Задание №2. Расчет совокупной аудитории ресурсов Avito и Ivi

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Avito+Ivi'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр по медиа, в нашем случае это ID ресурсов Avito и Ivi
mart_filter = 'crossMediaResourceId in (1067,1028)'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "useTypeName"]

# Указываем список статистик для расчета
statistics = ['reach', 'reachPer', 'adr', 'adrPer']

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter, demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_avito_ivi = mtask.result2table(mtask.get_result(task_audience), project_name)
df_avito_ivi

### Задание №3. Расчет аудитории Total Internet

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Total Internet'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии
demo_filter = None

# Задаем фильтр по медиа, в нашем случае он не требуется 
mart_filter = None

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "useTypeName"]

# Указываем список статистик для расчета
statistics = ['reach', 'reachPer', 'adr', 'adrPer']

scales = None

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('total', project_name, date_filter, usetype_filter, geo_filter,
                                      demo_filter, mart_filter, slices, statistics, scales)

# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_task(task_json))

# Получаем результат
df_total = mtask.result2table(mtask.get_result(task), project_name = project_name)
df_total

### Формирование итоговой таблицы

Следующим шагом соберем полученные результаты в один DataFrame

In [ ]:
# Объединим наши DataFrame'ы
df_result = pd.concat([df_avito, df_avito_ivi, df_total])
df_result

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('audience-avito-ivi'))
df_info = mtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()